# Knowledge Base
## Judul Topik
Efektivitas Imagery Rehearsal Therapy (IRT) dalam Mengurangi Nightmare Disorder, Gejala Depresi, Kecemasan, dan Ideasi Bunuh Diri pada Pasien Major Depressive Episode

## Ruang Lingkup Informasi
Knowledge base ini mencakup:
- Definisi dan Karakteristik Nightmare Disorder: Kriteria DSM-5-TR dan ICSD-3, prevalensi pada populasi umum vs psikiatrik
- Hubungan Nightmare dengan Depresi dan Risiko Bunuh Diri: Mekanisme bagaimana mimpi buruk memprediksi perilaku bunuh diri
- Imagery Rehearsal Therapy (IRT): Prinsip dasar, protokol 4 sesi, mekanisme kerja berdasarkan Continuity Theory
- Metodologi Penelitian: Desain studi non-randomized controlled, kriteria inklusi/eksklusi, instrumen pengukuran (NSI, QIDS-SR16, HAD, GAD-7)
- Hasil Klinis: Perbandingan efektivitas IRT vs Sleep Education Therapy (SET), effect sizes, responder rates
- Prediksi Respons Terapi: Faktor-faktor yang memprediksi keberhasilan IRT (treatment-resistant depression, frekuensi nightmare)
- Implikasi Klinis: Integrasi IRT dalam treatment plan pasien depresi dengan risiko bunuh diri tinggi
- 
## Alasan Pemilihan Topik
- Nightmare Disorder mempengaruhi 16.7% pasien dengan Major Depressive Episode (MDE), meningkat hingga 90% pada MDE dengan fitur melankolis atau perilaku bunuh diri
- Mimpi buruk adalah salah satu prediktor paling kuat untuk krisis bunuh diri, dengan 80% individu mengalami perubahan konten mimpi sebelum krisis bunuh diri
- IRT adalah first-line treatment untuk Nightmare Disorder sesuai rekomendasi internasional
- Studi ini merupakan first controlled study yang mengevaluasi IRT pada pasien MDE dengan Nightmare Disorder
- Menunjukkan effect sizes yang besar: Cohen's d = 1.33 untuk NSI, 0.95 untuk ideasi bunuh diri
- Menggunakan Nightmare Severity Index (NSI) yang mengukur 4 dimensi: frequency, emotional impact, diurnal impact, nocturnal impact
- Comprehensive evaluation mencakup depressive symptoms, anxiety, suicidal ideation, dan sleep quality
- Memahami progresivitas mimpi buruk → nightmare → skenario bunuh diri dapat membantu early intervention
- IRT dapat mencegah eskalasi gejala depresi menjadi krisis bunuh diri dengan menghentikan alterasi konten mimpi
- Temuan surprising bahwa treatment-resistant depression memprediksi respons lebih baik terhadap IRT
- Membuka perspektif baru untuk populasi yang sulit diobati dengan terapi konvensional

In [1]:
import os
import warnings
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

load_dotenv()

True

In [2]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY tidak ditemukan di environment")

print(" GROQ API Key berhasil dimuat dari environment")

 GROQ API Key berhasil dimuat dari environment


In [4]:
print("Loading embedding model...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print(" Embedding model loaded")

Loading embedding model...

 Embedding model loaded


In [5]:
from langchain_community.document_loaders import PyMuPDFLoader
import os

psikologi_dir = "dataset/Psikologi"

psikologi_pdfs = []
psikologi_docs = []
for file in os.listdir(psikologi_dir):
    if file.endswith(".pdf"):
        pdf_path = os.path.join(psikologi_dir, file)
        psikologi_pdfs.append(pdf_path)
        loader = PyMuPDFLoader(pdf_path)
        psikologi_docs.extend(loader.load())

print(f"Psikologi PDFs: {len(psikologi_pdfs)}")
print(f"Psikologi Documents: {len(psikologi_docs)}")

Psikologi PDFs: 1
Psikologi Documents: 11


In [6]:
display(psikologi_docs[:2] if psikologi_docs else "No history documents")

[Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'Elsevier', 'creationdate': '2025-12-22T18:45:42+00:00', 'source': 'dataset/Psikologi\\1-s2.0-S1697260025001152-main.pdf', 'file_path': 'dataset/Psikologi\\1-s2.0-S1697260025001152-main.pdf', 'total_pages': 11, 'format': 'PDF 1.7', 'title': 'Imagery Rehearsal Therapy (IRT) is associated with reduced nightmare severity and depressive, anxiety and suicidal symptoms in adults with Major Depressive Episode', 'author': 'Julia Maruani', 'subject': 'International Journal of Clinical and Health Psychology, 26 (2026) 100658. doi:10.1016/j.ijchp.2025.100658', 'keywords': '"Sleep disorders, Parasomnias, Psychotherapy, Major depressive episode",Suicidal Ideation,Controlled study', 'moddate': '2025-12-22T20:12:48+00:00', 'trapped': '', 'modDate': 'D:20251222201248Z', 'creationDate': 'D:20251222184542Z', 'page': 0}, page_content='Imagery Rehearsal Therapy (IRT) is associated with reduced nightmare \nseverity and depressi

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

psikologi_chunks = text_splitter.split_documents(psikologi_docs)

print(f"Psikologi chunks: {len(psikologi_chunks)}")

Psikologi chunks: 80


In [8]:
print("\nCreating vector store for clinical research...")
print(" Embedding research content...")

vector_store = FAISS.from_documents(psikologi_chunks, embeddings)

print(f" Vector store created with {vector_store.index.ntotal} vectors")

save_path = "faiss_irt_nightmare_depression"
vector_store.save_local(save_path)
print(f" Vector store saved to: {save_path}")


Creating vector store for clinical research...
 Embedding research content...
 Vector store created with 80 vectors
 Vector store saved to: faiss_irt_nightmare_depression


In [9]:
prompt_template = """Anda adalah asisten AI yang ahli dalam psikologi klinis dan psikiatri, khususnya tentang Imagery Rehearsal Therapy (IRT), Nightmare Disorder, dan Major Depressive Episode.

Gunakan konteks berikut untuk menjawab pertanyaan dengan akurat, detail, dan berdasarkan evidence-based research.

PENTING:
- Jika informasi tidak tersedia dalam konteks, katakan "Informasi tersebut tidak tersedia dalam artikel penelitian yang saya miliki."
- Jangan membuat informasi yang tidak ada dalam konteks.
- Untuk data statistik, kutip nilai p-value, Cohen's d, dan confidence intervals jika tersedia.
- Jelaskan implikasi klinis dari temuan penelitian.

Konteks Penelitian:
{context}

Pertanyaan: {question}

Jawaban berdasarkan evidence:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [10]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [11]:
retriever = vector_store.as_retriever(search_kwargs={"k": 4})

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | PROMPT
    | llm
    | StrOutputParser()
)

In [12]:
class ClinicalKnowledgeBaseManager:
    """Manager untuk update dan maintenance knowledge base psikologi klinis"""
    
    def __init__(self, vector_store, embeddings, text_splitter):
        self.vector_store = vector_store
        self.embeddings = embeddings
        self.text_splitter = text_splitter
        self.update_history = []
    
    def add_documents(self, new_documents: List[Document]):
        """Tambahkan dokumen baru ke knowledge base"""
        chunks = self.text_splitter.split_documents(new_documents)
        self.vector_store.add_documents(chunks)
        
        self.update_history.append({
            'timestamp': datetime.now().isoformat(),
            'action': 'add',
            'num_documents': len(new_documents),
            'num_chunks': len(chunks)
        })
        
        return f" Added {len(chunks)} chunks from {len(new_documents)} clinical research documents"
    
    def add_text(self, text: str, metadata: Dict = None):
        """Tambahkan teks langsung ke knowledge base"""
        doc = Document(page_content=text, metadata=metadata or {})
        return self.add_documents([doc])
    
    def add_clinical_article(self, pdf_path: str):
        """Tambahkan artikel penelitian klinis baru"""
        if not os.path.exists(pdf_path):
            return f" File not found: {pdf_path}"
        
        loader = PyMuPDFLoader(pdf_path)
        new_docs = loader.load()
        
        result = self.add_documents(new_docs)
        return f"{result}\n  Source: {os.path.basename(pdf_path)}"
    
    def rebuild_from_documents(self, documents: List[Document]):
        """Rebuild vector store dari awal dengan dokumen baru"""
        chunks = self.text_splitter.split_documents(documents)
        self.vector_store = FAISS.from_documents(chunks, self.embeddings)
        
        self.update_history.append({
            'timestamp': datetime.now().isoformat(),
            'action': 'rebuild',
            'num_documents': len(documents),
            'num_chunks': len(chunks)
        })
        
        return f" Rebuilt vector store with {len(chunks)} chunks"
    
    def save(self, path: str):
        """Simpan vector store ke disk"""
        self.vector_store.save_local(path)
        return f" Saved to {path}"
    
    def get_stats(self):
        """Dapatkan statistik knowledge base"""
        return {
            'total_vectors': self.vector_store.index.ntotal,
            'updates': len(self.update_history),
            'last_update': self.update_history[-1] if self.update_history else None
        }

kb_manager = ClinicalKnowledgeBaseManager(vector_store, embeddings, text_splitter)
print(" Clinical Knowledge Base Manager initialized")

 Clinical Knowledge Base Manager initialized


In [18]:
def ask_clinical_question(question: str, show_sources: bool = False):
    """Fungsi untuk menanyakan pertanyaan ke chatbot RAG psikologi klinis"""
    print(f"\n Clinical Question: {question}\n")
    print(" Retrieving evidence from research...")
    
    result = qa_chain.invoke({"query": question})
    
    print("="*80)
    print(" EVIDENCE-BASED ANSWER:")
    print("="*80)
    print(result['result'])
    print("="*80)
    
    if show_sources:
        print("\n SOURCE DOCUMENTS:")
        for i, doc in enumerate(result['source_documents'], 1):
            source = doc.metadata.get('source', 'unknown')
            page = doc.metadata.get('page', 'N/A')
            print(f"\n[{i}] {os.path.basename(source)} - Page {page}:")
            print(doc.page_content[:400] + "...")
    
    return result

print(" Clinical chatbot interface ready")

 Clinical chatbot interface ready


In [13]:
response1 = rag_chain.invoke(
    "Apa itu Nightmare Disorder dan berapa prevalensinya pada pasien dengan Major Depressive Episode?"
)

print(response1)

Nightmare Disorder adalah suatu kondisi di mana seseorang mengalami mimpi buruk yang sering dan parah, yang dapat menyebabkan gangguan tidur dan kesulitan dalam melakukan aktivitas sehari-hari. Menurut American Psychiatric Association (2013), Nightmare Disorder adalah suatu kondisi di mana seseorang mengalami mimpi buruk yang sering dan parah, yang dapat menyebabkan gangguan tidur dan kesulitan dalam melakukan aktivitas sehari-hari.

Prevalensi Nightmare Disorder pada pasien dengan Major Depressive Episode (MDE) bervariasi tergantung pada beberapa faktor, seperti jenis MDE dan adanya gejala-gejala lainnya. Menurut Akkaoui et al. (2020), prevalensi Nightmare Disorder pada pasien dengan MDE adalah sekitar 16,7%. Namun, prevalensi ini dapat meningkat hingga 90% jika pasien memiliki gejala-gejala lainnya seperti depresi melankolis atau perilaku bunuh diri.

Dalam penelitian yang disebutkan dalam konteks, prevalensi Nightmare Disorder pada pasien dengan MDE adalah sekitar 16,7%, yang mening

In [ ]:
response2 = rag_chain.invoke(
    "Bagaimana desain penelitian yang digunakan dalam studi IRT ini? Berapa jumlah sampel dan apa kriteria inklusinya?"
)

print(response2)

Desain penelitian yang digunakan dalam studi IRT ini adalah desain penelitian kontrol yang terkontrol. Penelitian ini membandingkan efektivitas Imagery Rehearsal Therapy (IRT) dengan Sleep Education Therapy (SET) pada pasien dewasa dengan Episode Depresi Utama (MDE).

Jumlah sampel dalam penelitian ini tidak disebutkan secara eksplisit dalam artikel penelitian yang saya miliki. Namun, dapat dilihat bahwa penelitian ini menggunakan dua kelompok: kelompok intervensi (IRT) dan kelompok kontrol (SET). Kelompok kontrol ini terdiri dari pasien yang menunggu intervensi IRT dan menerima SET.

Kriteria inklusi dalam penelitian ini tidak disebutkan secara eksplisit dalam artikel penelitian yang saya miliki. Namun, dapat dilihat bahwa penelitian ini menggunakan pasien dewasa dengan Episode Depresi Utama (MDE) sebagai sampel.

Dalam penelitian ini, pasien diukur pada dua waktu: waktu awal (baseline) dan waktu setelah intervensi (post-IRT). Penelitian ini juga menggunakan beberapa alat ukur, termas

In [15]:
response3 = rag_chain.invoke(
    "Seberapa efektif Imagery Rehearsal Therapy (IRT) dalam mengurangi nightmare severity dibandingkan Sleep Education Therapy (SET)? Sebutkan nilai p-value dan Cohen's d!"
)

print(response3)

Berdasarkan penelitian yang ada, Imagery Rehearsal Therapy (IRT) efektif dalam mengurangi nightmare severity dibandingkan Sleep Education Therapy (SET). Hasil penelitian menunjukkan bahwa IRT memiliki efek yang lebih signifikan dalam mengurangi nightmare severity, dengan nilai p-value < 0,001 dan Cohen's d = 1,3326.

Dalam penelitian tersebut, IRT juga menunjukkan perbaikan yang signifikan dalam semua dimensi nightmare severity, yaitu:

* Frekuensi: p = 0,010, d = 0,8452
* Dampak emosional: p = 0,003, d = 0,973
* Dampak siang hari: p = 0,017, d = 0,777
* Dampak malam hari: p = 0,002, d = 1,125

Hasil ini menunjukkan bahwa IRT lebih efektif dalam mengurangi nightmare severity dibandingkan SET.


In [16]:
response4 = rag_chain.invoke(
    "Apakah IRT juga efektif mengurangi gejala depresi, kecemasan, dan ideasi bunuh diri? Jelaskan dengan data statistik!"
)

print(response4)

Ya, IRT (Imagery Rehearsal Therapy) juga efektif mengurangi gejala depresi, kecemasan, dan ideasi bunuh diri. Menurut penelitian yang disebutkan dalam konteks, IRT terkait dengan penurunan keparahan gejala depresi, kecemasan, dan ideasi bunuh diri pada orang dewasa dengan Episode Depresi Utama (MDE).

Data statistik yang disebutkan dalam konteks adalah sebagai berikut:

* Penurunan gejala depresi: IRT terkait dengan penurunan gejala depresi yang signifikan (p < 0,01).
* Penurunan kecemasan: IRT terkait dengan penurunan kecemasan yang signifikan (p < 0,01).
* Penurunan ideasi bunuh diri: IRT terkait dengan penurunan ideasi bunuh diri yang signifikan (p < 0,01).

Namun, perlu diingat bahwa penelitian ini masih dalam tahap exploratory dan memiliki beberapa keterbatasan, seperti ukuran sampel yang terbatas dan timeline yang singkat. Oleh karena itu, perlu dilakukan penelitian lanjutan untuk memastikan efektivitas IRT dalam mengurangi gejala depresi, kecemasan, dan ideasi bunuh diri.

Impli

In [17]:
response5 = rag_chain.invoke(
    "Apa saja instrumen atau kuesioner yang digunakan untuk mengukur nightmare severity, depresi, dan kecemasan dalam penelitian ini?"
)

print(response5)

Dalam penelitian ini, beberapa instrumen atau kuesioner digunakan untuk mengukur nightmare severity, depresi, dan kecemasan. Berikut adalah beberapa instrumen yang disebutkan dalam artikel penelitian:

1. **Nightmare Severity Index (NSI)**: Instrumen ini digunakan untuk mengukur keparahan gangguan mimpi buruk (nightmare disorder). NSI mencakup empat dimensi utama gangguan mimpi buruk, yaitu:
 * Frekuensi (frequency)
 * Dampak emosional (emotional impact)
 * Dampak harian (diurnal impact)
 * Dampak malam (nocturnal impact)
2. **Structured Clinical Interview for DSM-5-TR**: Instrumen ini digunakan untuk mendiagnosis gangguan mimpi buruk dan episode depresi mayor (major depressive episode) berdasarkan kriteria DSM-5-TR.
3. **Insomnia Severity Index (ISI)**: Instrumen ini digunakan untuk mengukur keparahan insomnia.
4. **Kuesioner kecemasan**: Meskipun tidak disebutkan secara spesifik, artikel penelitian menyebutkan bahwa penelitian ini juga mengukur kecemasan sebagai salah satu hasil yang

In [18]:
response6 = rag_chain.invoke(
    "Faktor apa saja yang memprediksi keberhasilan terapi IRT? Apakah treatment-resistant depression berpengaruh?"
)

print(response6)

Berdasarkan artikel penelitian yang tersedia, faktor yang memprediksi keberhasilan terapi IRT adalah treatment-resistant depression (TRD). Penelitian menemukan bahwa pasien dengan TRD yang menerima terapi IRT menunjukkan peningkatan yang lebih signifikan dalam mengurangi gejala depresi, kecemasan, dan ideasi bunuh diri dibandingkan dengan pasien yang tidak memiliki TRD.

Hasil penelitian menunjukkan bahwa pasien dengan TRD yang menerima terapi IRT memiliki penurunan yang signifikan dalam skala depresi (p < 0,01), kecemasan (p < 0,05), dan ideasi bunuh diri (p < 0,01) dibandingkan dengan pasien yang tidak memiliki TRD.

Penelitian juga menemukan bahwa pasien dengan TRD yang menerima terapi IRT memiliki peningkatan yang signifikan dalam mengurangi keparahan mimpi buruk (p < 0,01) dan mengurangi gejala insomnia (p < 0,05).

Dalam konteks klinis, hasil penelitian ini menunjukkan bahwa terapi IRT dapat menjadi alternatif yang efektif untuk pasien dengan TRD yang tidak responsif terhadap ter

In [19]:
response7 = rag_chain.invoke(
    "Bagaimana mekanisme Imagery Rehearsal Therapy bekerja dalam mengurangi nightmare dan suicidal ideation?"
)

print(response7)

Mekanisme Imagery Rehearsal Therapy (IRT) dalam mengurangi nightmare dan suicidal ideation dapat dijelaskan sebagai berikut:

1. **Transformasi Emosi dalam Mimpi**: IRT membantu pasien mengubah emosi yang mengganggu dalam mimpi mereka menjadi lebih positif melalui proses kognitif dan perilaku restrukturisasi. Dengan demikian, pasien dapat mengurangi dampak emosional yang negatif dari mimpi mereka, yang kemudian dapat mempengaruhi suasana hati mereka pada siang hari.
2. **Pengurangan Dampak Emosional Mimpi**: Penelitian menunjukkan bahwa dampak emosional mimpi lebih kuat terkait dengan depresi dan ideasi bunuh diri daripada frekuensi mimpi sendiri (Ellis et al., 2019). Oleh karena itu, IRT dapat membantu mengurangi dampak emosional mimpi, yang kemudian dapat mengurangi risiko depresi dan ideasi bunuh diri.
3. **Pengurangan Depresi**: IRT juga dapat membantu mengurangi gejala depresi, yang merupakan salah satu prediktor kuat ideasi bunuh diri (Thünker & Pietrowsky, 2012). Dengan menguran

In [20]:
response8 = rag_chain.invoke(
    "Jelaskan protokol 4 sesi Imagery Rehearsal Therapy yang digunakan dalam penelitian ini!"
)

print(response8)

Dalam penelitian ini, protokol 4 sesi Imagery Rehearsal Therapy (IRT) yang digunakan adalah sebagai berikut:

1. **Sesi 1**: Pasien menerima penjelasan tentang IRT dan teknik mental imagery. Mereka juga diminta untuk memahami konsep IRT dan bagaimana cara menggunakannya untuk mengurangi keparahan mimpi buruk.
2. **Sesi 2**: Pasien diminta untuk memulai latihan mental imagery secara mandiri di rumah. Mereka harus merekam mimpi buruk mereka dalam sebuah buku harian dan menggambarkan mimpi buruk tersebut dengan cara yang lebih positif. Pasien juga diminta untuk melakukan latihan mental imagery beberapa kali sehari.
3. **Sesi 3**: Pasien menerima umpan balik dari terapis tentang kemajuan mereka dalam melakukan latihan mental imagery. Mereka juga diminta untuk meningkatkan intensitas latihan mental imagery dan untuk memperhatikan perubahan-perubahan yang terjadi dalam mimpi buruk mereka.
4. **Sesi 4**: Pasien menerima umpan balik final dari terapis tentang kemajuan mereka dalam melakukan IR

In [21]:
response9 = rag_chain.invoke(
    "Apa saja keterbatasan atau limitasi dari penelitian ini?"
)

print(response9)

Berdasarkan konteks penelitian yang disediakan, beberapa keterbatasan atau limitasi dari penelitian ini adalah:

1. **Desain penelitian**: Penelitian ini memiliki desain penelitian yang terbatas, yaitu hanya memiliki dua titik waktu, yaitu baseline dan post-intervensi. Penelitian yang lebih panjang dan memiliki lebih banyak titik waktu dapat memberikan informasi yang lebih akurat tentang efek jangka panjang dari terapi IRT.
2. **Jumlah sampel**: Jumlah sampel penelitian ini terbatas, yaitu hanya 53 orang. Penelitian yang memiliki jumlah sampel yang lebih besar dapat memberikan hasil yang lebih akurat dan dapat dieneralisir ke populasi yang lebih luas.
3. **Penggunaan skala tunggal**: Penelitian ini menggunakan skala tunggal untuk mengukur gejala depresi, kecemasan, dan ide suicidal. Penggunaan skala tunggal dapat memiliki keterbatasan dalam mengukur gejala yang kompleks ini. Penelitian yang menggunakan skala yang lebih komprehensif, seperti Columbia-Suicide Severity Rating Scale (C-SSR

In [22]:
response10 = rag_chain.invoke(
    "Apa implikasi klinis dari temuan penelitian ini untuk praktik psikologi dan psikiatri?"
)

print(response10)

Temuan penelitian ini menunjukkan bahwa Imagery Rehearsal Therapy (IRT) dapat mengurangi keparahan gangguan mimpi buruk dan gejala depresi, kecemasan, dan ideasi bunuh diri pada orang dewasa dengan Episode Depresi Mayor (MDE). Selain itu, IRT juga dapat meningkatkan efisiensi tidur, durasi tidur, dan kekhawatiran insomnia.

Implikasi klinis dari temuan penelitian ini adalah:

1. **Penggunaan IRT sebagai terapi tambahan**: Hasil penelitian ini menunjukkan bahwa IRT dapat menjadi terapi tambahan yang efektif untuk mengurangi gejala depresi, kecemasan, dan ideasi bunuh diri pada pasien dengan MDE. Dengan demikian, IRT dapat menjadi salah satu pilihan terapi yang dapat digunakan bersama dengan terapi lainnya.
2. **Pengurangan keparahan gangguan mimpi buruk**: Penelitian ini menunjukkan bahwa IRT dapat mengurangi keparahan gangguan mimpi buruk pada pasien dengan MDE. Hal ini dapat membantu pasien untuk memiliki kualitas tidur yang lebih baik dan mengurangi gejala depresi dan kecemasan.
3. *